# 导数和微分

## 1.几个简单的例子

**导数***

(1) 生成数据x，令 requires_grad=True

(2) 建立 x 与 y的关系

(3) 求解导数 backward()

(4) reset 梯度 x.grad.zero_()

In [36]:
import torch

In [37]:
# requires_grad:是否需要计算梯度并保留
x = torch.arange(4.0, requires_grad=True)
x, x.grad #默认梯度为 0

(tensor([0., 1., 2., 3.], requires_grad=True), None)

In [38]:
y = 2 * torch.dot(x, x) # y=2*x的平方, y是一个具体的数值
y

tensor(28., grad_fn=<MulBackward0>)

In [39]:
y.backward()
x.grad #导数相当于 y = 4x，带入tensor([0., 1., 2., 3.]验证
x

tensor([0., 1., 2., 3.], requires_grad=True)

## 2. 非标量的反向传播 

In [42]:
#在深度学习中，有时候y是矩阵，而不是一个数字，此时需要y.sum()成一个数值
x.grad.zero_()
y = x * x
y.sum().backward() #或者 y.backward(gradient=torch.ones(len(y)))
x.grad

tensor([0., 2., 4., 6.])

## 3. detach 计算

In [55]:
'''
引入变量保留不需要计算梯度的中间项，假设我们有 z = x * y 和 y = x * x，但我们希望关注 x 对 z 的直接影响
而不是通过 y 传递的影响。 在这种情况下，我们可以创建一个新变量 u，它的值与 y 相同，
但它的出处（它是如何创建的）已被删除。 因此 u 在图中没有祖先，并且梯度不会通过 u 流向 x。
例如，采用 z = x * u 的梯度将产生结果 x
'''
x.grad.zero_()
y = x * x
u = y.detach() # u 复制 y 的值，但 u 不保存梯度
z = u * x
z.sum().backward()
x, y, x.grad

(tensor([0., 1., 2., 3.], requires_grad=True),
 tensor([0., 1., 4., 9.], grad_fn=<MulBackward0>),
 tensor([0., 1., 4., 9.]))

In [56]:
'''
虽然此过程将 y 的祖先从通向 z 的图中分离，但通向 y 的计算图仍然存在，因此我们可以计算 y 相对于 x 的梯度。
'''
x.grad.zero_()
y.sum().backward()
x.grad

tensor([0., 2., 4., 6.])